In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas import DataFrame, Series
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler

from datatools import extract_market_data, data_quantization
from pipeline import Dataset
from pipeline.backtest import evaluation_for_submission, cross_validation
from visualization.metric import plot_performance

import xarray as xr

In [2]:
base_ds = xr.open_dataset('../../data/nc/base.nc')
market_brief_ds = xr.open_dataset('../../data/nc/market_brief.nc')
ds = base_ds.merge(market_brief_ds)

In [7]:
current_day = 550
before1_day = 549
train_lookback = 16
per_eval_lookback = 8
X = ds.sel(day=range(current_day - train_lookback - per_eval_lookback, before1_day))[['close_0', 'open_0', 'pe']]
y = ds.sel(day=range(current_day - train_lookback - 1, before1_day))['return']

In [8]:
X

<xarray.Dataset>
Dimensions:  (day: 23, asset: 54)
Coordinates:
  * day      (day) int32 526 527 528 529 530 531 532 ... 543 544 545 546 547 548
  * asset    (asset) int32 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53
Data variables:
    close_0  (day, asset) float64 ...
    open_0   (day, asset) float64 ...
    pe       (day, asset) float64 ...

In [9]:
y

<xarray.DataArray 'return' (day: 16, asset: 54)>
[864 values with dtype=float64]
Coordinates:
  * day      (day) int32 533 534 535 536 537 538 539 ... 543 544 545 546 547 548
  * asset    (asset) int32 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53

In [6]:
start_day = X.day.min().item()

526

In [15]:
xr.concat([X.sel(day=slice(start_day + i, start_day + i + per_eval_lookback - 1)).expand_dims(
    batch=[start_day + i + per_eval_lookback - 1]) for i in range(train_lookback)], dim='batch'
).stack(batch_asset=['batch', 'asset'], create_index=True).to_array('feature').transpose('batch_asset', 'day',
                                                                                         'feature')

<xarray.Dataset>
Dimensions:  (day: 23, asset: 54, batch: 16)
Coordinates:
  * day      (day) int32 526 527 528 529 530 531 532 ... 543 544 545 546 547 548
  * asset    (asset) int32 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53
  * batch    (batch) int64 533 534 535 536 537 538 ... 543 544 545 546 547 548
Data variables:
    close_0  (batch, day, asset) float64 21.13 27.33 19.15 ... 5.547 23.89 14.55
    open_0   (batch, day, asset) float64 21.52 27.04 19.52 ... 5.571 23.63 14.67
    pe       (batch, day, asset) float64 25.16 36.5 38.14 ... 44.65 39.27 21.08

In [23]:
X_list = []
for i in range(train_lookback):
    X_slice = X.sel(day=slice(start_day + i, start_day + i + per_eval_lookback - 1)).expand_dims(
        batch=[start_day + i + per_eval_lookback - 1])
    X_slice.coords['offset'] = X_slice.day - start_day - i
    X_slice_o = X_slice.swap_dims({'day': 'offset'})
    X_list.append(X_slice_o.reset_coords(drop=True))
X_concat = xr.concat(X_list, dim='batch')
X_concat

<xarray.Dataset>
Dimensions:  (batch: 16, offset: 8, asset: 54)
Coordinates:
  * batch    (batch) int64 533 534 535 536 537 538 ... 543 544 545 546 547 548
  * asset    (asset) int32 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53
  * offset   (offset) int32 0 1 2 3 4 5 6 7
Data variables:
    close_0  (batch, offset, asset) float64 21.13 27.33 19.15 ... 23.89 14.55
    open_0   (batch, offset, asset) float64 21.52 27.04 19.52 ... 23.63 14.67
    pe       (batch, offset, asset) float64 25.16 36.5 38.14 ... 39.27 21.08

In [24]:
X_concat.isnull().any()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    close_0  bool False
    open_0   bool False
    pe       bool False

In [30]:
X_arr = X_concat.stack({'batch_asset': ['batch', 'asset']}).to_array('feature').transpose('batch_asset', 'offset', 'feature')

In [31]:
X_arr.values

array([[[21.1323, 21.5207, 25.1564],
        [21.2536, 21.108 , 25.3113],
        [21.108 , 21.108 , 25.1282],
        ...,
        [21.2172, 20.8288, 25.269 ],
        [21.2294, 21.2415, 25.2831],
        [21.0594, 21.2051, 25.0718]],

       [[27.3348, 27.0435, 36.5048],
        [27.2377, 27.007 , 36.3827],
        [27.444 , 27.4076, 36.6491],
        ...,
        [28.6578, 28.1237, 38.2807],
        [28.9127, 28.67  , 38.6136],
        [28.2815, 28.937 , 37.7701]],

       [[19.1538, 19.5179, 38.1431],
        [19.348 , 19.1416, 38.5318],
        [19.9185, 19.3237, 39.668 ],
        ...,
        [20.1734, 20.1612, 40.1664],
        [19.9185, 20.1734, 39.668 ],
        [19.8699, 19.9185, 39.5683]],

       ...,

       [[ 5.6684,  5.6563, 45.6377],
        [ 5.6442,  5.6684, 45.4732],
        [ 5.5835,  5.6078, 44.9798],
        ...,
        [ 5.5956,  5.6078, 45.062 ],
        [ 5.5592,  5.5956, 44.7331],
        [ 5.5471,  5.5713, 44.6509]],

       [[23.7783, 23.8269, 39.0692],
  

In [32]:
X_arr.batch_asset

<xarray.DataArray 'batch_asset' (batch_asset: 864)>
array([(533, 0), (533, 1), (533, 2), ..., (548, 51), (548, 52), (548, 53)],
      dtype=object)
Coordinates:
  * batch_asset  (batch_asset) object MultiIndex
  * batch        (batch_asset) int64 533 533 533 533 533 ... 548 548 548 548 548
  * asset        (batch_asset) int32 0 1 2 3 4 5 6 7 ... 46 47 48 49 50 51 52 53

In [47]:
y_pred_np = y.stack({'batch_asset': ['day', 'asset']}).values ** 2

In [51]:
from xarray import DataArray

y_pred_da = DataArray(data=y_pred_np, coords=dict(batch_asset=X_arr.batch_asset)).unstack('batch_asset').rename({'batch': 'day'})
y_pred_da

<xarray.DataArray (day: 16, asset: 54)>
array([[3.40146851e-04, 1.65834545e-04, 1.49561973e-06, 1.20303132e-03,
        3.62776209e-04, 1.88712349e-04, 2.26577693e-04, 2.28705613e-05,
        3.51909388e-05, 3.99692485e-04, 6.71101659e-04, 6.16313340e-05,
        2.94774611e-04, 1.99291539e-04, 6.52202838e-05, 2.11031485e-05,
        6.67691108e-05, 2.94695557e-04, 3.45186200e-05, 7.62176656e-05,
        1.78891560e-04, 1.75644919e-05, 2.01530122e-04, 3.34943583e-05,
        8.21578328e-05, 6.99009342e-04, 2.27304188e-05, 4.77529240e-04,
        4.70014296e-05, 7.98021852e-04, 7.83260883e-05, 2.77887600e-03,
        4.08357290e-06, 3.32298240e-04, 1.26969730e-04, 1.97581513e-05,
        3.34233171e-04, 2.19365653e-05, 1.49342684e-05, 2.52323768e-05,
        1.74428156e-04, 2.75671585e-04, 5.58553608e-05, 8.50253041e-05,
        8.20739410e-04, 2.64370752e-04, 1.27314018e-04, 5.41395776e-04,
        2.19244025e-04, 1.03215969e-03, 6.93601228e-04, 4.08852043e-05,
        5.19812317e-03, 3.50370503e-03],
       [9.97002878e-05, 1.21924248e-04, 1.54341276e-03, 5.51354159e-03,
        5.74237023e-06, 3.35128742e-07, 6.87912801e-04, 1.17654436e-04,
        5.13473564e-05, 7.45447833e-04, 1.30077161e-03, 7.53360820e-04,
        7.66137116e-04, 1.18762919e-03, 3.02215153e-05, 4.05813006e-04,
        4.73169621e-05, 7.60955831e-05, 2.58507872e-04, 1.02982980e-02,
        3.07982006e-03, 8.98051169e-04, 2.04515851e-03, 5.32074659e-04,
...
        7.02018199e-04, 7.98358416e-04, 3.26284700e-05, 1.17311295e-03,
        7.09991860e-07, 2.32741300e-04, 5.66046599e-06, 9.24623058e-04,
        1.78596392e-04, 6.92071014e-06, 4.93824243e-03, 2.69838355e-05,
        1.49348931e-05, 2.47300660e-04, 8.73458102e-06, 3.51243745e-05,
        1.55088622e-03, 2.58238081e-03, 2.54062518e-06, 1.71489748e-04,
        1.66754229e-05, 2.50646617e-04],
       [4.19402514e-04, 9.06041762e-04, 1.11506608e-03, 1.11845035e-03,
        2.79331103e-04, 3.23744602e-04, 6.61291251e-04, 1.18747613e-04,
        2.04901059e-04, 0.00000000e+00, 2.63964211e-04, 3.27176479e-04,
        7.94358564e-03, 2.08214497e-04, 2.18650624e-04, 8.07792382e-04,
        4.66726562e-05, 1.29218250e-02, 9.85190717e-04, 1.37163747e-03,
        1.01485946e-03, 2.49494787e-06, 9.34853727e-05, 8.49405025e-04,
        7.73842380e-04, 1.81984704e-04, 4.51250388e-04, 2.20489081e-04,
        5.30997851e-05, 1.02570450e-03, 7.57203306e-04, 5.68147248e-04,
        1.64055177e-03, 7.99358112e-05, 3.25038930e-05, 5.48914016e-04,
        4.02798853e-05, 3.17814908e-04, 2.28837855e-05, 7.50546338e-04,
        2.21004568e-04, 5.66979917e-04, 4.78793938e-03, 3.05145433e-04,
        9.82992364e-04, 1.12648232e-06, 2.21369416e-04, 5.69617527e-06,
        1.49947260e-03, 3.97224258e-03, 2.85432001e-03, 4.30596779e-05,
        1.03482643e-06, 6.68194170e-04]])
Coordinates:
  * day      (day) int64 533 534 535 536 537 538 539 ... 543 544 545 546 547 548
  * asset    (asset) int64 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53

<xarray.DataArray (day: 16, asset: 54)>
array([[3.40146851e-04, 1.65834545e-04, 1.49561973e-06, 1.20303132e-03,
        3.62776209e-04, 1.88712349e-04, 2.26577693e-04, 2.28705613e-05,
        3.51909388e-05, 3.99692485e-04, 6.71101659e-04, 6.16313340e-05,
        2.94774611e-04, 1.99291539e-04, 6.52202838e-05, 2.11031485e-05,
        6.67691108e-05, 2.94695557e-04, 3.45186200e-05, 7.62176656e-05,
        1.78891560e-04, 1.75644919e-05, 2.01530122e-04, 3.34943583e-05,
        8.21578328e-05, 6.99009342e-04, 2.27304188e-05, 4.77529240e-04,
        4.70014296e-05, 7.98021852e-04, 7.83260883e-05, 2.77887600e-03,
        4.08357290e-06, 3.32298240e-04, 1.26969730e-04, 1.97581513e-05,
        3.34233171e-04, 2.19365653e-05, 1.49342684e-05, 2.52323768e-05,
        1.74428156e-04, 2.75671585e-04, 5.58553608e-05, 8.50253041e-05,
        8.20739410e-04, 2.64370752e-04, 1.27314018e-04, 5.41395776e-04,
        2.19244025e-04, 1.03215969e-03, 6.93601228e-04, 4.08852043e-05,
        5.19812317e-03, 3.50370503e-03],
       [9.97002878e-05, 1.21924248e-04, 1.54341276e-03, 5.51354159e-03,
        5.74237023e-06, 3.35128742e-07, 6.87912801e-04, 1.17654436e-04,
        5.13473564e-05, 7.45447833e-04, 1.30077161e-03, 7.53360820e-04,
        7.66137116e-04, 1.18762919e-03, 3.02215153e-05, 4.05813006e-04,
        4.73169621e-05, 7.60955831e-05, 2.58507872e-04, 1.02982980e-02,
        3.07982006e-03, 8.98051169e-04, 2.04515851e-03, 5.32074659e-04,
...
        7.02018199e-04, 7.98358416e-04, 3.26284700e-05, 1.17311295e-03,
        7.09991860e-07, 2.32741300e-04, 5.66046599e-06, 9.24623058e-04,
        1.78596392e-04, 6.92071014e-06, 4.93824243e-03, 2.69838355e-05,
        1.49348931e-05, 2.47300660e-04, 8.73458102e-06, 3.51243745e-05,
        1.55088622e-03, 2.58238081e-03, 2.54062518e-06, 1.71489748e-04,
        1.66754229e-05, 2.50646617e-04],
       [4.19402514e-04, 9.06041762e-04, 1.11506608e-03, 1.11845035e-03,
        2.79331103e-04, 3.23744602e-04, 6.61291251e-04, 1.18747613e-04,
        2.04901059e-04, 0.00000000e+00, 2.63964211e-04, 3.27176479e-04,
        7.94358564e-03, 2.08214497e-04, 2.18650624e-04, 8.07792382e-04,
        4.66726562e-05, 1.29218250e-02, 9.85190717e-04, 1.37163747e-03,
        1.01485946e-03, 2.49494787e-06, 9.34853727e-05, 8.49405025e-04,
        7.73842380e-04, 1.81984704e-04, 4.51250388e-04, 2.20489081e-04,
        5.30997851e-05, 1.02570450e-03, 7.57203306e-04, 5.68147248e-04,
        1.64055177e-03, 7.99358112e-05, 3.25038930e-05, 5.48914016e-04,
        4.02798853e-05, 3.17814908e-04, 2.28837855e-05, 7.50546338e-04,
        2.21004568e-04, 5.66979917e-04, 4.78793938e-03, 3.05145433e-04,
        9.82992364e-04, 1.12648232e-06, 2.21369416e-04, 5.69617527e-06,
        1.49947260e-03, 3.97224258e-03, 2.85432001e-03, 4.30596779e-05,
        1.03482643e-06, 6.68194170e-04]])
Coordinates:
  * day      (day) int64 533 534 535 536 537 538 539 ... 543 544 545 546 547 548
  * asset    (asset) int64 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53